In [0]:
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# ==============================================================================
# Configuração & Carga
# ==============================================================================
TABLE_NAME_SILVER = "silver_climate_data"

# Carrega a tabela Prata tratada
df_silver = spark.read.table(TABLE_NAME_SILVER)

print(f"Dados carregados da tabela '{TABLE_NAME_SILVER}' com sucesso.")

# ==============================================================================
# Análises de Negócio
# ==============================================================================

# ------------------------------------------------------------------------------
# Pergunta 1: Tendência Temporal
# "Qual é a tendência de variação da Temperatura Média global e por continente?"
# ------------------------------------------------------------------------------
def analyze_temperature_trend(df: DataFrame) -> DataFrame:
    """
    Agrupa por Ano e Continente para analisar a evolução da temperatura.
    """
    return df.groupBy("year", "continent") \
             .agg(F.avg("avg_temperature_celsius").alias("avg_temp_global")) \
             .orderBy("year", "continent")

df_trend = analyze_temperature_trend(df_silver)
print("--- Análise 1: Tendência Temporal (Amostra) ---")
display(df_trend) 


# ------------------------------------------------------------------------------
# Pergunta 2: Correlação Crítica
# "Existe correlação entre Emissões de CO2 e Aumento do Nível do Mar ou Temperatura?"
# ------------------------------------------------------------------------------
def calculate_correlations(df: DataFrame) -> None:
    """
    Calcula a correlação de Pearson entre as métricas críticas.
    """
    corr_co2_temp = df.stat.corr("co2_emissions_mt", "avg_temperature_celsius")
    corr_co2_sea  = df.stat.corr("co2_emissions_mt", "sea_level_rise_mm")
    
    print("--- Análise 2: Correlações Estatísticas (Pearson) ---")
    print(f"Correlação CO2 vs Temperatura: {corr_co2_temp:.4f}")
    print(f"Correlação CO2 vs Nível do Mar: {corr_co2_sea:.4f}")
    
    data = [("CO2 vs Temp", corr_co2_temp), ("CO2 vs Sea Level", corr_co2_sea)]
    display(spark.createDataFrame(data, ["metric_pair", "correlation_coefficient"]))

calculate_correlations(df_silver)


# ------------------------------------------------------------------------------
# Pergunta 3: Análise de Vulnerabilidade
# "Top 5 países com maior Climate Risk Index comparados à média global"
# ------------------------------------------------------------------------------
def analyze_vulnerability(df: DataFrame) -> DataFrame:
    """
    Identifica os Top 5 países mais vulneráveis e compara seus indicadores
    com as médias globais do dataset.
    """
    # 1. Calcular Médias Globais
    global_avgs = df.select(
        F.avg("avg_temperature_celsius").alias("global_avg_temp"),
        F.avg("sea_level_rise_mm").alias("global_avg_sea")
    ).collect()[0]
    
    # 2. Filtrar Top 5 Risco (considerando o ano mais recente ou média geral)
    # Aqui vamos pegar a média histórica do país para ser mais justo
    df_country_risk = df.groupBy("country") \
                        .agg(
                            F.avg("climate_risk_index").alias("avg_risk_index"),
                            F.avg("avg_temperature_celsius").alias("country_avg_temp"),
                            F.avg("sea_level_rise_mm").alias("country_avg_sea")
                        ) \
                        .orderBy(F.desc("avg_risk_index")) \
                        .limit(5)
    
    # 3. Adicionar comparação
    df_comparison = df_country_risk \
        .withColumn("global_avg_temp", F.lit(global_avgs["global_avg_temp"])) \
        .withColumn("diff_temp_from_global", F.col("country_avg_temp") - F.col("global_avg_temp")) \
        .withColumn("global_avg_sea", F.lit(global_avgs["global_avg_sea"]))
        
    return df_comparison

df_vulnerability = analyze_vulnerability(df_silver)
print("--- Análise 3: Top 5 Países Vulneráveis vs Média Global ---")
display(df_vulnerability)


# ------------------------------------------------------------------------------
# Pergunta 4: Impacto Regional
# "Qual continente tem a maior média de emissões e como é sua temperatura?"
# ------------------------------------------------------------------------------
def analyze_regional_impact(df: DataFrame) -> DataFrame:
    """
    Agrega dados por continente para identificar os maiores emissores.
    """
    return df.groupBy("continent") \
             .agg(
                 F.avg("co2_emissions_mt").alias("avg_co2_emissions"),
                 F.avg("avg_temperature_celsius").alias("avg_temperature")
             ) \
             .orderBy(F.desc("avg_co2_emissions"))

df_regional = analyze_regional_impact(df_silver)
print("--- Análise 4: Impacto Regional (Emissões vs Temperatura) ---")
display(df_regional)
